This notebook contains a re-implementation of the "global" phonetiser from Brian Ó Raghallaigh's Ph.D. thesis using [rbg2p](https://github.com/stts-se/rbg2p).



Brian Ó Raghallaigh (2010). *Multi-dialect phonetisation for Irish text-to-speech synthesis: a modular approach.* (Doctoral thesis, Trinity College, Dublin), Appendix B.1

```
@phdthesis{oraghallaigh2010multidialect,
  author       = {Brian Ó~Raghallaigh}, 
  title        = {Multi-dialect phonetisation for {I}rish text-to-speech synthesis: a modular approach},
  school       = {Trinity College, Dublin},
  year         = 2010,
  address      = {Dublin, Ireland},
  month        = 9,
}
```

The initial run (after small conversion errors were corrected) gave the following set of errors:

```
FAILED TEST: for 'comhair', expected /K OO RJ/, got /K OMH RJ/
FAILED TEST: for 'airde', expected /AA RJ DJ @@/, got /AA R DJ @@/
FAILED TEST: for 'bogha', expected /B OGH/, got /B OGH @@/
FAILED TEST: for 'comhar', expected /K OO R/, got /K OMH R/
FAILED TEST: for 'ceird', expected /KJ EE RJ DJ/, got /KJ EE R DJ/
FAILED TEST: for 'riail', expected /RJ I@ LJ/, got /R I@ LJ/
FAILED TEST: for 'giuirléid', expected /GJ UU RJ LJ EE DJ/, got /GJ UU R LJ EE DJ/
FAILED TEST: for 'boird', expected /B OO RJ DJ/, got /B OO R DJ/
FAILED TEST: for 'bantaboic', expected /B A NN T @@ B @@ KJ/, got /B A NN T @@ B OI KJ/
FAILED TEST: for 'comhar', expected /K OO R/, got /K OMH R/
FAILED TEST: for 'bantaboc', expected /B A NN T @@ B @@ K/, got /B A NN T @@ B O K/
FAILED TEST: for 'guird', expected /G UU RJ DJ/, got /G UU R DJ/
FAILED TEST: for 'bhfianaise', expected /VJ I@ NN @@ SJ @@/, got /V I@ NN @@ SJ @@/
FAILED TEST: for 'leathbhosca', expected /LJ A V @@ S K @@/, got /LJ A V O S K @@/
FAILED TEST: for 'rithfeadh', expected /RJ I XJ ADH/, got /R I XJ ADH/
FAILED TEST: for 'tsagairt', expected /T A G @@ RJ TJ/, got /T A G @@ R TJ/
16 OF 168 TESTS FAILED FOR briain.g2p
exit status 1
```

The most consistent source of errors is slender 'r' in environments (word initially, before 'd', etc.) where the rule is that they should be left broad; I corrected the tests (which were intended only as tests of the vowels).

Of the remainder, _bogha_ represents a missing rule, while _comhar/comhair_ does not fit with the given rule, so I added a somewhat lexicalised rule to handle it (and its mutated forms) specifically.

This leaves these words:
```
FAILED TEST: for 'bantaboic', expected /B A NN T @@ B @@ KJ/, got /B A NN T @@ B OI KJ/
FAILED TEST: for 'bantaboc', expected /B A NN T @@ B @@ K/, got /B A NN T @@ B O K/
FAILED TEST: for 'leathbhosca', expected /LJ A V @@ S K @@/, got /LJ A V O S K @@/
```

_leathbhosca_ is a compound, and keeping _bhosca_ as `V O S K @@` is correct; otherwise, the rule which for other short vowels converts to schwa is specifically converted to `O`, so I disabled these rules.

In [ ]:
%%writefile oraghallaigh.g2p
CHARACTER_SET "aábcdeéfghiíjklmnoópqrstuúvwxyz'-’"
DEFAULT_PHONEME "_"
PHONEME_DELIMITER " "

#VAR ALPHA [abcdefghijklmnopqrstuvwxyz]
VAR CONS [bcdfghjklmnpqrstvwxyz]
VAR CONSS [bcdfghjklmnpqrstvwxyz]*
VAR CONSP [bcdfghjklmnpqrstvwxyz]+
# including '#' doesn't work, so those rules were duplicated
#VAR NONSYLLABIC (á|b|c|d|f|g|h|j|k|l|m|n|ó|p|q|r|s|t|ú|v|w|x|y|z|#)
VAR NONSYLLABIC [ábcdfghjklmnópqrstúvwxyz]

// broad future/conditional endings
VAR BFCE (á|adh|aidh|aidís|aimid|aimis|ainn|as)
// slender future/conditional endings
VAR SFCE (eá|eadh|idh|idís|imid|imis|inn)
#VAR L (ll|l)
#VAR MN [mn]
VAR FMP [fmp]
#VAR LNR [lnr]
VAR LNRP [lnr]+
#VAR LRST [lrst]
VAR DNLST [dnlst]
#VAR DNST [dnst]
VAR RDNLR (rd|rn|rl|rr)
VAR VOWEL [aáeéiíoóuú]
VAR VOWELS [aáeéiíoóuú]*
VAR VOWELP [aáeéiíoóuú]+
// left context short broad vowel
VAR LCSBV (ea|io|iu|a|o|u)
// left context short slender vowel
VAR LCSSV (ai|eai|ei|e|iui|i|oi|ui)
// left context broad vowel
VAR LCBV (adh|ae|ao|aá|ea|eá|eo|éa|io|iu|iú|ío|oó|uío|ua|u|ú)
// right context broad vowel
#VAR RCBV (aei|ae|ai|aoi|ao|a|ái|á|oi|o|ói|ó|ui|uío|uí|u|úi|ú)
// left context slender vowel
VAR LCSV (aei|aidh|ai|aí|aoi|ái|eai|eái|ei|eoi|e|éi|é|iai|iui|iúi|i|í|oi|ói|uai|ui|uí|úi)
VAR LCSVS (aei|aidh|ai|aí|aoi|ái|eai|eái|ei|eoi|e|éi|é|iai|iui|iúi|i|í|oi|ói|uai|ui|uí|úi)*
// right context slender vowel
VAR RCSV (eai|ea|eái|eá|ei|eoi|eo|e|éa|éi|é|iai|ia|io|iui|iu|iúi|iú|i|ío|í)
// left context long vowel
VAR LCLV (aei|ae|aoi|ao|ái|á|eái|eá|eoi|eó|eo|éi|é|iúi|iú|ío|í|ói|ó|uío|uí|úi|ú)
// left context slender long vowel
VAR LCSLV (aei|aidh|aoi|ái|eái|eoi|éi|é|iúi|í|ói|uai|uí|úi)

ádh -> AA
ái -> AA
á -> AA

abh -> ABH
adh -> ADH / _ #
adh -> AI
agh -> AI
aei -> EE
ae -> EE
aíodh -> ÍODH / _ #
aío -> AÍO
aí -> II
aidh -> IDH / _ #
aidh -> AI
aigh -> IGH / _ #
aigh -> AI
aithe -> ITHE / _ #
ai -> ∅ / # CONSS VOWELS abh _ CONSP
ai -> ∅ / # CONSS VOWELS adh _ CONSP
ai -> ∅ / # CONSS VOWELS agh _ CONSP
ai -> ∅ / # CONSS VOWELS amh _ CONSP
ai -> ∅ / # CONSS obh _ CONSP
ai -> ∅ / # CONSS VOWELS ódh _ CONSP
ai -> ∅ / # CONSS odh _ CONSP
ai -> ∅ / # CONSS VOWELS ogh _ CONSP
ai -> ∅ / # CONSS VOWELS omh _ CONSP
ai -> ∅ / # CONSS VOWELS umh _ CONSP

ai -> AA / # CONSS _ RDNLR
ai -> A / # CONSS _
ai -> @@ / VOWELP CONSP _
ai -> A
amh -> AMH / _ #
amh -> AU
aoi -> AO
ao -> AO

a -> ∅ / # CONSS VOWELS abh _ CONSP
a -> ∅ / # CONSS VOWELS adh _ CONSP
a -> ∅ / # CONSS VOWELS agh _ CONSP
a -> ∅ / # CONSS VOWELS amh _ CONSP
a -> ∅ / # CONSS obh _ CONSP
a -> ∅ / # CONSS VOWELS ódh _ CONSP
a -> ∅ / # CONSS odh _ CONSP
a -> ∅ / # CONSS VOWELS ogh _ CONSP
a -> ∅ / # CONSS VOWELS omh _ CONSP
a -> ∅ / # CONSS VOWELS umh _ CONSP

# addition
a -> ∅ / # CONSS VOWELS ogh _ #
omh -> OO / (gc|ch|c) _ (ai|a) r

a -> AA / # CONSS _ RDNLR
a -> A / # CONSS _

a -> @@ / VOWELP CONSP _
a -> A

éa -> EE
éi -> EE
é -> EE

eái -> AA
eá -> AA
eabh -> ABH
eadh -> ADH / _ #
eadh -> AU
eagh -> AI
eai -> A
eamh -> AMH / _ #
eamh -> AU / # CONSS _
# VOWEL, or VOWELS ??
ea -> ∅ / # CONSS VOWEL igh _ CONSP

ea -> AA / # CONSS _ RDNLR
ea -> A / # CONSS _

ea -> @@ / VOWELP CONSP _
ea -> A

eidh -> EIDH
eigh -> EIGH

ei -> EE / # CONSS _ RDNLR #
ei -> EE / # CONSS _ RDNLR NONSYLLABIC
ei -> E / # CONSS _
ei -> E
eódh -> OO
eoi -> OO
eó -> OO
eo -> OO
e -> E / # CONSS _
e -> @@ / VOWELP CONSP _
e -> E

íodh -> ÍODH / _ #
ío -> II
í -> II

iadh -> I@
iath -> I@
iai -> I@
ia -> I@
idh -> IDH
igh -> IGH
io -> IO
ithe -> ITHE / _ #
iúi -> UU
iú -> UU
iubh -> UBH
iumh -> UU
iui -> UU
iu -> U
i -> ∅ / # CONSS VOWEL idh _ CONSP #
i -> ∅ / # CONSS VOWEL igh _ CONSP
i -> @@ / VOWELP CONSP _
i -> I

ódh -> ÓDH / _ #
ói -> OO
ó -> OO

obh -> OBH
odh -> ODH
ogh -> OGH

oí -> II
oidh -> OIDH
oigh -> OIGH

oi -> OO / # CONSS _ RDNLR
oi -> @@ / # VOWELP CONSP _
oi -> OI
omh -> OMH

o -> OO / # CONSS _ RDNLR #
o -> OO / # CONSS _ RDNLR NONSYLLABIC
o -> O / # VOWELP CONSP _
o -> O

úi -> UU
ú -> UU

uath -> U@
uai -> U@
ua -> U@
ubh -> UBH
ue -> E

ui -> UU / # CONSS _ RDNLR #
ui -> UU / # CONSS _ RDNLR NONSYLLABIC
ui -> I / # CONSS _
ui -> @@ / VOWELP CONSP _
ui -> UI
uío -> II
uí -> II
umh -> UU

u -> UU / # CONSS _ RDNLR
u -> U / # CONSS _
u -> @@ / VOWELP CONSP _
u -> U

bf -> P / _ BFCE #
bf -> PJ / _ SFCE #
bhf -> VJ / # _ CONSS RCSV #
bhf -> VJ / # _ CONSS RCSV NONSYLLABIC
bhf -> V / # _
bhf -> F / _ BFCE #
bhf -> FJ / _ SFCE #
bh -> @@ V / # LCSBV LNRP _
bh -> @@ V / NONSYLLABIC LCSBV LNRP _
bh -> @@ VJ / # LCSSV LNRP _
bh -> @@ VJ / NONSYLLABIC LCSSV LNRP _

bh -> VJ / # _ CONSS RCSV #
bh -> VJ / # _ CONSS RCSV NONSYLLABIC
bh -> V / # _

bh -> VJ / _ CONSS RCSV #
bh -> VJ / _ CONSS RCSV NONSYLLABIC
bh -> VJ / # LCSV CONSP _
bh -> VJ / NONSYLLABIC LCSV CONSP _
bh -> VJ / # LCSV _
bh -> VJ / NONSYLLABIC LCSV _
bh -> V
 
bp -> BJ / # _ CONSS RCSV #
bp -> BJ / # _ CONSS RCSV NONSYLLABIC
bp -> B / # _

bth -> P / LCBV CONSS _
bth -> PJ / LCSV CONSS _

b -> @@ B / # LCSBV LNRP _
b -> @@ B / NONSYLLABIC LCSBV LNRP _
b -> @@ BJ / # LCSSV LNRP _
b -> @@ BJ / NONSYLLABIC LCSSV LNRP _

b -> BJ / _ CONSS RCSV #
b -> BJ / _ CONSS RCSV NONSYLLABIC
b -> BJ / # LCSV CONSP _
b -> BJ / NONSYLLABIC LCSV CONSP _
b -> BJ / # LCSV _
b -> BJ / NONSYLLABIC LCSV _
b -> B

cf -> K / _ BFCE #
cf -> KJ / _ SFCE #

chf -> X / _ BFCE #
chf -> XJ / _ SFCE #

ch -> @@ X / # LCSBV LNRP _
ch -> @@ X / NONSYLLABIC LCSBV LNRP _
ch -> @@ XJ / # LCSSV LNRP _
ch -> @@ XJ / NONSYLLABIC LCSSV LNRP _

ch -> XJ / # _ CONSS RCSV #
ch -> XJ / # _ CONSS RCSV NONSYLLABIC
ch -> X / # _

ch -> XJ / _ CONSS RCSV #
ch -> XJ / _ CONSS RCSV NONSYLLABIC
ch -> XJ / # LCSV CONSP _
ch -> XJ / NONSYLLABIC LCSV CONSP _
ch -> XJ / # LCSV _
ch -> XJ / NONSYLLABIC LCSV _
ch -> X
 
cth -> K / LCBV CONSS _
cth -> KJ / LCSV CONSS _

c -> KJ / _ CONSS RCSV #
c -> KJ / _ CONSS RCSV NONSYLLABIC
c -> KJ / # LCSV CONSP _
c -> KJ / NONSYLLABIC LCSV CONSP _
c -> KJ / # LCSV _
c -> KJ / NONSYLLABIC LCSV _
c -> K

df -> T / _ BFCE #
df -> TJ / _ SFCE #

dha -> ∅ / # LCLV _
dha -> ∅ / NONSYLLABIC LCLV _
dh -> GFJ / # LCSLV _
dh -> GFJ / NONSYLLABIC LCSLV _
dh -> GFJ / # CONSS LCSVS _ #
dh -> ∅ / # LCLV _
dh -> ∅ / NONSYLLABIC LCLV _

dh -> GFJ / # _ CONSS RCSV #
dh -> GFJ / # _ CONSS RCSV NONSYLLABIC
dh -> GF / # _

dh -> GFJ / _ CONSS RCSV #
dh -> GFJ / _ CONSS RCSV NONSYLLABIC
dh -> GFJ / # LCSV CONSP _
dh -> GFJ / NONSYLLABIC LCSV CONSP _
dh -> GFJ / # LCSV _
dh -> GFJ / NONSYLLABIC LCSV _
dh -> GF

dt -> DJ / # _ CONSS RCSV #
dt -> DJ / # _ CONSS RCSV NONSYLLABIC
dt -> D / # _

dt -> T / LCBV CONSS _
dt -> TJ / LCSV CONSS _

d -> DJ / _ CONSS RCSV #
d -> DJ / _ CONSS RCSV NONSYLLABIC
d -> DJ / # LCSV CONSP _
d -> DJ / NONSYLLABIC LCSV CONSP _
d -> DJ / # LCSV _
d -> DJ / NONSYLLABIC LCSV _
d -> D

fh -> ∅
f -> H / VOWEL _ BFCE #
f -> HJ / VOWEL _ SFCE #

f -> @@ F / # LCSBV LNRP _
f -> @@ F / NONSYLLABIC LCSBV LNRP _
f -> @@ FJ / # LCSSV LNRP _
f -> @@ FJ / NONSYLLABIC LCSSV LNRP _

f -> FJ / _ CONSS RCSV #
f -> FJ / _ CONSS RCSV NONSYLLABIC
f -> FJ / # LCSV CONSP _
f -> FJ / NONSYLLABIC LCSV CONSP _
f -> FJ / # LCSV _
f -> FJ / NONSYLLABIC LCSV _
f -> F

gc -> GJ / # _ CONSS RCSV #
gc -> GJ / # _ CONSS RCSV NONSYLLABIC
gc -> G / # _

gf -> K / _ BFCE #
gf -> KJ / _ SFCE #

gh -> GFJ / # _ CONSS RCSV #
gh -> GFJ / # _ CONSS RCSV NONSYLLABIC
gh -> GF / # _

gh -> GFJ / # LCSLV _
gh -> GFJ / NONSYLLABIC LCSLV _
gh -> GFJ / # CONSS LCSVS _ #
gh -> ∅ / # LCLV _
gh -> ∅ / NONSYLLABIC LCLV _

gh -> GFJ / _ CONSS RCSV #
gh -> GFJ / _ CONSS RCSV NONSYLLABIC
gh -> GFJ / # LCSV CONSP _
gh -> GFJ / NONSYLLABIC LCSV CONSP _
gh -> GFJ / # LCSV _
gh -> GFJ / NONSYLLABIC LCSV _
gh -> GF

gth -> K / LCBV CONSS _
gth -> KJ / LCSV CONSS _

g -> @@ G / # LCSBV LNRP _
g -> @@ G / NONSYLLABIC LCSBV LNRP _
g -> @@ GJ / # LCSSV LNRP _
g -> @@ GJ / NONSYLLABIC LCSSV LNRP _

g -> GJ / _ CONSS RCSV #
g -> GJ / _ CONSS RCSV NONSYLLABIC
g -> GJ / # LCSV CONSP _
g -> GJ / NONSYLLABIC LCSV CONSP _
g -> GJ / # LCSV _
g -> GJ / NONSYLLABIC LCSV _
g -> G

h -> HJ / _ CONSS RCSV #
h -> HJ / _ CONSS RCSV NONSYLLABIC
h -> HJ / # LCSV CONSP _
h -> HJ / NONSYLLABIC LCSV CONSP _
h -> HJ / # LCSV _
h -> HJ / NONSYLLABIC LCSV _
h -> H

j -> DJZJ

k -> KJ / _ CONSS RCSV #
k -> KJ / _ CONSS RCSV NONSYLLABIC
k -> KJ / # LCSV CONSP _
k -> KJ / NONSYLLABIC LCSV CONSP _
k -> KJ / # LCSV _
k -> KJ / NONSYLLABIC LCSV _
k -> K

llf -> LL_D / _ BFCE #
llf -> LLJ_D / _ SFCE #

llth -> LL_D / LCBV CONSS _
llth -> LLJ_D / LCSV CONSS _

ll -> LLJ / _ CONSS RCSV #
ll -> LLJ / _ CONSS RCSV NONSYLLABIC
ll -> LLJ / # LCSV CONSP _
ll -> LLJ / NONSYLLABIC LCSV CONSP _
ll -> LLJ / # LCSV _
ll -> LLJ / NONSYLLABIC LCSV _
ll -> LL

lf -> LL_D / _ BFCE #
lf -> LJ_D / _ SFCE #

lth -> LL_D / LCBV CONSS _
lth -> LJ_D / LCSV CONSS _

l -> LJ / _ CONSS RCSV #
l -> LJ / _ CONSS RCSV NONSYLLABIC
l -> LJ / # LCSV CONSP _
l -> LJ / NONSYLLABIC LCSV CONSP _
l -> LJ / # LCSV _
l -> LJ / NONSYLLABIC LCSV _
l -> LL

mb -> MJ / # _ CONSS RCSV #
mb -> MJ / # _ CONSS RCSV NONSYLLABIC
mb -> M / # _

mf -> M_D / _ BFCE #
mf -> MJ_D / _ SFCE #

mhf -> F / _ BFCE #
mhf -> FJ / _ SFCE #

mh -> VJ / # _ CONSS RCSV #
mh -> VJ / # _ CONSS RCSV NONSYLLABIC
mh -> V / # _

mh -> @@ V / # LCSBV LNRP _
mh -> @@ V / NONSYLLABIC LCSBV LNRP _
mh -> @@ VJ / # LCSSV LNRP _
mh -> @@ VJ / NONSYLLABIC LCSSV LNRP _

mh -> VJ / _ CONSS RCSV #
mh -> VJ / _ CONSS RCSV NONSYLLABIC
mh -> VJ / # LCSV CONSP _
mh -> VJ / NONSYLLABIC LCSV CONSP _
mh -> VJ / # LCSV _
mh -> VJ / NONSYLLABIC LCSV _
mh -> V

mth -> M_D / LCBV CONSS _
mth -> MJ_D / LCSV CONSS _

m -> @@ M / # LCSBV LNRP _
m -> @@ M / NONSYLLABIC LCSBV LNRP _
m -> @@ MJ / # LCSSV LNRP _
m -> @@ MJ / NONSYLLABIC LCSSV LNRP _

m -> MJ / _ CONSS RCSV #
m -> MJ / _ CONSS RCSV NONSYLLABIC
m -> MJ / # LCSV CONSP _
m -> MJ / NONSYLLABIC LCSV CONSP _
m -> MJ / # LCSV _
m -> MJ / NONSYLLABIC LCSV _
m -> M

nnf -> NN_D / _ BFCE #
nnf -> NNJ_D / _ SFCE #

nnth -> NN_D / LCBV CONSS _
nnth -> NNJ_D / LCSV CONSS _

nn -> NNJ / _ CONSS RCSV #
nn -> NNJ / _ CONSS RCSV NONSYLLABIC
nn -> NNJ / # LCSV CONSP _
nn -> NNJ / NONSYLLABIC LCSV CONSP _
nn -> NNJ / # LCSV _
nn -> NNJ / NONSYLLABIC LCSV _
nn -> NN

n- -> NJ / # _ RCSV #
n- -> NJ / # _ RCSV NONSYLLABIC
n- -> NN / # _

nd -> NNJ / # _ CONSS RCSV #
nd -> NNJ / # _ CONSS RCSV NONSYLLABIC
nd -> NN / # _

nf -> NN_D / _ BFCE #
nf -> NJ_D / _ SFCE #

ngf -> NG_D / _ BFCE #
ngf -> NGJ_D / _ SFCE #

ngth -> NG_D / LCBV CONSS _
ngth -> NGJ_D / LCSV CONSS _

ng -> NGJ / # _ CONSS RCSV #
ng -> NGJ / # _ CONSS RCSV NONSYLLABIC
ng -> NG / # _
ng -> NJ / # LCSV _ t #
ng -> NJ / NONSYLLABIC LCSV _ t #

ng -> NGJ / _ CONSS RCSV #
ng -> NGJ / _ CONSS RCSV NONSYLLABIC
ng -> NGJ / # LCSV CONSP _
ng -> NGJ / NONSYLLABIC LCSV CONSP _
ng -> NGJ / # LCSV _
ng -> NGJ / NONSYLLABIC LCSV _
ng -> NG

nth -> NN_D / LCBV CONSS _
nth -> NJ_D / LCSV CONSS _

n -> NGJ / # LCSV _ c
n -> NGJ / NONSYLLABIC LCSV _ c
n -> NG / _ c

n -> NJ / _ CONSS RCSV #
n -> NJ / _ CONSS RCSV NONSYLLABIC
n -> NJ / # LCSV CONSP _
n -> NJ / NONSYLLABIC LCSV CONSP _
n -> NJ / # LCSV _
n -> NJ / NONSYLLABIC LCSV _
n -> NN

pf -> P / _ BFCE #
pf -> PJ / _ SFCE #

ph -> FJ / # _ CONSS RCSV #
ph -> FJ / # _ CONSS RCSV NONSYLLABIC
ph -> F / # _

ph -> FJ / _ CONSS RCSV #
ph -> FJ / _ CONSS RCSV NONSYLLABIC
ph -> FJ / # LCSV CONSP _
ph -> FJ / NONSYLLABIC LCSV CONSP _
ph -> FJ / # LCSV _
ph -> FJ / NONSYLLABIC LCSV _
ph -> F

pth -> P / LCBV CONSS _
pth -> PJ / LCSV CONSS _

p -> PJ / _ CONSS RCSV #
p -> PJ / _ CONSS RCSV NONSYLLABIC
p -> PJ / # LCSV CONSP _
p -> PJ / NONSYLLABIC LCSV CONSP _
p -> PJ / # LCSV _
p -> PJ / NONSYLLABIC LCSV _
p -> P

# really? there's a 'W' in the phoneset
q -> K V

rrf -> RR_D / _ BFCE #
rrf -> RRJ_D / _ SFCE #

rrth -> RR_D / LCBV CONSS _
rrth -> RRJ_D / LCSV CONSS _

rr -> RRJ / _ CONSS RCSV #
rr -> RRJ / _ CONSS RCSV NONSYLLABIC
rr -> RRJ / # LCSV CONSP _
rr -> RRJ / NONSYLLABIC LCSV CONSP _
rr -> RRJ / # LCSV _
rr -> RRJ / NONSYLLABIC LCSV _
rr -> RR

rf -> R_D / _ BFCE #
rf -> RJ_D / _ SFCE #

rth -> R_D / LCBV CONSS _
rth -> RJ_D / LCSV CONSS _

r -> R / # s _
r -> R / # _
// This rule blocks tests for airde and ceird
r -> R / _ DNLST

r -> RJ / _ CONSS RCSV #
r -> RJ / _ CONSS RCSV NONSYLLABIC
r -> RJ / # LCSV CONSP _
r -> RJ / NONSYLLABIC LCSV CONSP _
r -> RJ / # LCSV _
r -> RJ / NONSYLLABIC LCSV _
r -> R

sf -> S / _ BFCE #
sf -> SJ / _ SFCE #

shl -> LJ_D / _ CONSS RCSV #
shl -> LJ_D / _ CONSS RCSV NONSYLLABIC
shl -> LL_D
shm -> MJ_D / _ CONSS RCSV #
shm -> MJ_D / _ CONSS RCSV NONSYLLABIC
shm -> M_D
shn -> NJ_D / _ CONSS RCSV #
shn -> NJ_D / _ CONSS RCSV NONSYLLABIC
shn -> NN_D
shr -> RJ_D / _ CONSS RCSV #
shr -> RJ_D / _ CONSS RCSV NONSYLLABIC
shr -> R_D

sh -> XJ / # _ CONSS RCSV #
sh -> XJ / # _ CONSS RCSV NONSYLLABIC
sh -> H / # _

sh -> XJ / _ CONSS RCSV #
sh -> XJ / _ CONSS RCSV NONSYLLABIC
sh -> XJ / # LCSV CONSP _
sh -> XJ / NONSYLLABIC LCSV CONSP _
sh -> XJ / # LCSV _
sh -> XJ / NONSYLLABIC LCSV _
sh -> H

s -> S / # _ r
s -> S / # _ FMP CONSS RCSV #
s -> S / # _ FMP CONSS RCSV NONSYLLABIC

s -> SJ / _ CONSS RCSV #
s -> SJ / _ CONSS RCSV NONSYLLABIC
s -> SJ / # LCSV CONSP _
s -> SJ / NONSYLLABIC LCSV CONSP _
s -> SJ / # LCSV _
s -> SJ / NONSYLLABIC LCSV _
s -> S

t- -> TJ / # _ RCSV #
t- -> TJ / # _ RCSV NONSYLLABIC
t- -> T / # _

tf -> T / _ BFCE #
tf -> TJ / _ SFCE #

// "hack for compound boundaries"
th -> ∅ / _ CONS h

thb -> PJ / _ CONSS RCSV #
thb -> PJ / _ CONSS RCSV NONSYLLABIC
thb -> P
thc -> KJ / _ CONSS RCSV #
thc -> KJ / _ CONSS RCSV NONSYLLABIC
thc -> K
thd -> TJ / _ CONSS RCSV #
thd -> TJ / _ CONSS RCSV NONSYLLABIC
thd -> T

thf -> H / _ BFCE #
thf -> XJ / _ SFCE #

thl -> LJ_D / _ CONSS RCSV #
thl -> LJ_D / _ CONSS RCSV NONSYLLABIC
thl -> LL_D
thm -> MJ_D / _ CONSS RCSV #
thm -> MJ_D / _ CONSS RCSV NONSYLLABIC
thm -> M_D
thn -> NJ_D / _ CONSS RCSV #
thn -> NJ_D / _ CONSS RCSV NONSYLLABIC
thn -> NN_D
thp -> PJ / _ CONSS RCSV #
thp -> PJ / _ CONSS RCSV NONSYLLABIC
thp -> P
thr -> RJ_D / _ CONSS RCSV #
thr -> RJ_D / _ CONSS RCSV NONSYLLABIC
thr -> R_D
ths -> SJ / _ CONSS RCSV #
ths -> SJ / _ CONSS RCSV NONSYLLABIC
ths -> S
tht -> TJ / _ CONSS RCSV #
tht -> TJ / _ CONSS RCSV NONSYLLABIC
tht -> T

th -> HJ / # _ CONSS RCSV #
th -> HJ / # _ CONSS RCSV NONSYLLABIC
th -> H / # _

th -> HJ / _ CONSS RCSV #
th -> HJ / _ CONSS RCSV NONSYLLABIC
th -> HJ / # LCSV CONSP _
th -> HJ / NONSYLLABIC LCSV CONSP _
th -> HJ / # LCSV _
th -> HJ / NONSYLLABIC LCSV _
th -> H

ts -> TJ / # _ CONSS RCSV #
ts -> TJ / # _ CONSS RCSV NONSYLLABIC
ts -> T / # _

t -> TJ / _ CONSS RCSV #
t -> TJ / _ CONSS RCSV NONSYLLABIC
t -> TJ / # LCSV CONSP _
t -> TJ / NONSYLLABIC LCSV CONSP _
t -> TJ / # LCSV _
t -> TJ / NONSYLLABIC LCSV _
t -> T

v -> VJ / _ CONSS RCSV #
v -> VJ / _ CONSS RCSV NONSYLLABIC
v -> VJ / # LCSV CONSP _
v -> VJ / NONSYLLABIC LCSV CONSP _
v -> VJ / # LCSV _
v -> VJ / NONSYLLABIC LCSV _
v -> V

w -> V

x- -> E KJ S
x -> ZJ

y -> GFJ

z -> ZJ / _ CONSS RCSV #
z -> ZJ / _ CONSS RCSV NONSYLLABIC
z -> ZJ / # LCSV CONSP _
z -> ZJ / NONSYLLABIC LCSV CONSP _
z -> ZJ / # LCSV _
z -> ZJ / NONSYLLABIC LCSV _
z -> Z

' -> ∅
’ -> ∅
- -> ∅

TEST ádh -> AA
TEST áiseanna -> AA SJ @@ NN @@
TEST áthas -> AA H @@ S
TEST abhainn -> ABH NNJ
TEST bualadh -> B U@ LL ADH
TEST sadhbh -> S AI V
TEST saghas -> S AI S
TEST gaeilge -> G EE LJ GJ @@
TEST saolaíodh -> S AO LL ÍODH
TEST gardaí -> G AA R D II
TEST dúnfaidh -> D UU NN_D IDH
TEST aidhm -> AI MJ
TEST cheadaigh -> XJ A D IGH
TEST aighneas -> AI NJ @@ S
TEST diúltaithe -> DJ UU LL T ITHE
TEST seabhaic -> SJ ABH KJ
TEST feadhain -> FJ AU NJ
TEST teaghais -> TJ AI SJ
TEST eamhain -> AU NJ
TEST lobhair -> LL OBH RJ
TEST leódhais -> LJ OO SJ
TEST bodhair -> B ODH RJ
TEST eoghain -> OO NJ
TEST broghais -> B R OGH SJ
TEST comhair -> K OO RJ
TEST ciumhais -> KJ UU SJ
# wiktionary has: ˈɑːɾˠdʲə ˈiːɾˠdʲə ˈɑːɾˠdʲə ˈaiɾʲdʲə ˈæːɾˠdʲə ˈʌɾˠdʲə, so rule seems right
#TEST airde -> AA RJ DJ @@
TEST airde -> AA R DJ @@
TEST cait -> K A TJ
TEST sodair -> S O D @@ RJ
TEST ait -> A TJ
TEST déanamh -> DJ EE NN AMH
TEST amharc -> AU R K
TEST gaoil -> G AO LJ
TEST gaol -> G AO LL
TEST seabhac -> SJ ABH K
TEST ceadharlach -> KJ AU R LL @@ X
TEST teaghasán -> TJ AI S AA NN
TEST lobhar -> LL OBH R
TEST leódhas -> LJ OO S
TEST bodhar -> B ODH R
TEST eoghan -> OO NN
TEST bogha -> B OGH
TEST comhar -> K OO R
TEST dumhach -> D UU X
TEST ard -> AA R D
TEST cat -> K A T
TEST sodar -> S O D @@ R
TEST at -> A T
TEST éan -> EE NN
TEST éiníní -> EE NJ II NJ II
TEST é -> EE
TEST sheáin -> XJ AA NJ
TEST seán -> SJ AA NN
TEST seabhac -> SJ ABH K
TEST seinneadh -> SJ E NNJ ADH
TEST ceadharlach -> KJ AU R LL @@ X
TEST teaghlach -> TJ AI LL @@ X
TEST beairic -> BJ A RJ @@ KJ
TEST áireamh -> AA RJ AMH
TEST sleamhnán -> SJ LJ AU NN AA NN
TEST oighear -> OIGH R
TEST ceard -> KJ AA R D
TEST cead -> KJ A D
TEST áireamhán -> AA RJ @@ V AA NN
TEST eas -> A S
TEST feidhm -> FJ EIDH MJ
TEST leigheas -> LJ EIGH S
# wiktionary gives cəiɾˠdʲ and cɪɾˠdʲ; the rule seems right
#TEST ceird -> KJ EE RJ DJ
TEST ceird -> KJ EE R DJ
TEST deis -> DJ E SJ
TEST eitpheil -> E TJ FJ E LJ
TEST ceoil -> KJ OO LJ
TEST bainseó -> B A NJ SJ OO
TEST ceol -> KJ OO LL
TEST uile -> I LJ @@
TEST ceannaíodh -> KJ A NN ÍODH
TEST síos -> SJ II S
TEST sí -> SJ II
TEST siadhail -> SJ I@ LJ
TEST sciath -> SJ KJ I@
#TEST riail -> RJ I@ LJ
TEST riail -> R I@ LJ
TEST siad -> SJ I@ D
TEST seinnfidh -> SJ E NNJ_D IDH
TEST cheannaigh -> XJ A NN IGH
TEST fios -> FJ IO S
TEST imithe -> I MJ ITHE
TEST siúil -> SJ UU LJ
TEST siúl -> SJ UU LL
TEST tiubh -> TJ UBH
TEST ciumhais -> KJ UU SJ
#TEST giuirléid -> GJ UU RJ LJ EE DJ
TEST giuirléid -> GJ UU R LJ EE DJ
TEST fiuch -> FJ U X
TEST leighis -> LJ EIGH SJ
TEST foighid -> F OIGH DJ
TEST aithris -> A RJ_D @@ SJ
TEST sin -> SJ I NJ
TEST cheannódh -> XJ A NN ÓDH
TEST óil -> OO LJ
TEST ól -> OO LL
TEST lobhadh -> LL OBH ADH
TEST todhchaí -> T ODH X II
TEST toghadh -> T OGH ADH
TEST oíche -> II XJ @@
TEST oidhe -> OIDH @@
TEST oighear -> OIGH R
#TEST boird -> B OO RJ DJ
TEST boird -> B OO R DJ
TEST soir -> S OI RJ
TEST comhar -> K OO R
TEST bord -> B OO R D
TEST bos -> B O S
TEST súil -> S UU LJ
TEST súl -> S UU LL
TEST uathúil -> U@ UU LJ
TEST uaine -> U@ NJ @@
TEST uan -> U@ NN
TEST subh -> S UBH
// not LJ ?
TEST bhuel -> V E LL
#TEST guird -> G UU RJ DJ
TEST guird -> G UU R DJ
TEST cuid -> K I DJ
TEST uile -> I LJ @@
TEST bruíon -> B R II NN
TEST bruíne -> B R II NJ @@
TEST cumhacht -> K UU X T
TEST burdún -> B UU R D UU NN
TEST cur -> K U R
TEST bus -> B U S
TEST scuabfaidh -> S K U@ P IDH
TEST clibfidh -> KJ LJ I PJ IDH
TEST scuabfadh -> S K U@ P ADH
TEST clibfeadh -> KJ LJ I PJ ADH
TEST bhfáinne -> V AA NNJ @@
TEST bhfianaise -> VJ I@ NN @@ SJ @@
TEST scríobhfaidh -> SJ KJ RJ II F IDH
TEST díbhfidh -> DJ II FJ IDH
TEST scríobhfadh -> SJ KJ RJ II F ADH
TEST díbhfeadh -> DJ II FJ ADH
TEST searbh -> SJ A R @@ V
TEST seirbhís -> SJ E RJ @@ VJ II SJ
TEST bhrostaigh -> V R O S T IGH
TEST bhris -> VJ RJ I SJ
TEST coibhín -> K OI VJ II NJ
TEST bpáistí -> B AA SJ TJ II
TEST bpéisteanna -> BJ EE SJ TJ @@ NN @@
TEST scuabtha -> S K U@ P @@
TEST clibthe -> KJ LJ I PJ @@
TEST borb -> B O R @@ B
TEST seirbiach -> SJ E RJ @@ BJ I@ X
TEST bróna -> B R OO NN @@
TEST brian -> BJ RJ I@ NN
TEST leódhas -> LJ OO S
TEST t-uisce -> T UI SJ KJ @@
TEST t-éabhlóidí -> TJ EE V LL OO DJ II
TEST atfaidh -> A T IDH
TEST titfidh -> TJ I TJ IDH
TEST athdhéanamh -> A GFJ EE NN AMH
TEST meathfadh -> MJ A H ADH
#TEST rithfeadh -> RJ I XJ ADH
TEST rithfeadh -> R I XJ ADH
TEST bláthra -> B LL AA R_D @@
TEST tharla -> H AA R LL @@
TEST thit -> HJ I TJ
TEST tseachtain -> TJ A X T @@ NJ
#TEST tsagairt -> T A G @@ RJ TJ
# wiktionary (sagairt): ˈsˠaɡəɾˠtʲ ˈsˠæɡəɾˠtʲ
TEST tsagairt -> T A G @@ R TJ
TEST teann -> TJ A NN
TEST tit -> TJ I TJ
TEST togra -> T O G R @@
TEST sadhbh -> S AI V

# disabled; the rule for 'o' in an unaccented syllable
# does not produce schwa; also, 'leathbhosca' is a compound;
# the 'o' should not be reduced
#TEST leathbhosca -> LJ A V @@ S K @@
#TEST bantaboic -> B A NN T @@ B @@ KJ
#TEST bantaboc -> B A NN T @@ B @@ K

TEST bhéal -> VJ EE LL
TEST bhéil -> VJ EE LJ